In [2]:
import medmnist
from medmnist import INFO
from medmnist.dataset import DermaMNIST

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import dataset_without_pytorch
from dataset_without_pytorch import get_loader

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

import cnn


In [3]:
data_flag = 'dermamnist'
download = True

BATCH_SIZE = 128 #Lote tamanio

info_28 = INFO[data_flag] # Info

# Obtener dinamicamente la clase del conjunto de datos desde un modulo
DataClass_28 = getattr(dataset_without_pytorch, info_28['python_class'])

train_dataset_28 = DataClass_28(split='train', download=download)

# Encapsular los datos en un DataLoader para hacer lotes
train_loader_28 = get_loader(dataset=train_dataset_28, batch_size=BATCH_SIZE)

Using downloaded and verified file: C:\Users\abiab\.medmnist\dermamnist.npz


In [4]:
images = train_dataset_28.imgs
labels = train_dataset_28.labels

images_class_5 = [image for image, label in zip(images, labels) if label == 5]

In [5]:
flattened_images = [image.flatten() for image in images_class_5]

# Aplicar K-means con 1000 clusters
kmeans = KMeans(n_clusters=1000, random_state=42)
kmeans.fit(flattened_images)

# Obtener los centros de los clusters
cluster_centers = kmeans.cluster_centers_

# Para cada centroide, encontrar la imagen más cercana en el conjunto original
selected_images = []
for center in cluster_centers:
    # Calcular la distancia entre el centroide y todas las imágenes
    distances = np.linalg.norm(flattened_images - center, axis=1)
    # Encontrar el índice de la imagen más cercana
    closest_image_index = np.argmin(distances)
    # Añadir la imagen más cercana a la lista de imágenes seleccionadas
    selected_images.append(images_class_5[closest_image_index])

c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [6]:
# Filtrar imágenes y etiquetas de todas las clases excepto la clase 5
images_other_classes = [image for image, label in zip(images, labels) if label != 5]
labels_other_classes = [label for label in labels if label != 5]

# Crear etiquetas para las imágenes seleccionadas de la clase 5
selected_labels = [5] * len(selected_images)

# Combinar las imágenes y etiquetas de las otras clases con las seleccionadas de la clase 5
new_images = images_other_classes + selected_images

selected_labels = [[x] for x in selected_labels]

new_labels = labels_other_classes + selected_labels

In [7]:
new_labels = [int(label[0]) if isinstance(label, (list, np.ndarray)) else int(label) for label in new_labels]

new_images = np.array(new_images)
new_labels = np.array(new_labels)

# Calcular la distribución de clases
unique, counts = np.unique(new_labels, return_counts=True)
class_distribution = dict(zip(unique, counts))

# Calcular el promedio de imágenes por clase
average_images = int(np.mean(list(class_distribution.values())))

# Crear un generador de datos para rotación
datagen = ImageDataGenerator(rotation_range=40)

# Listas para las imágenes y etiquetas aumentadas
augmented_images = []
augmented_labels = []

# Aumentar imágenes en las clases 3 y 6 hasta alcanzar el promedio
target_classes = [3, 6]
for target_class in target_classes:
    class_images = new_images[new_labels == target_class]
    current_count = class_distribution[target_class]
    images_to_add = average_images - current_count

    if images_to_add > 0:
        for i in range(images_to_add):
            # Seleccionar una imagen aleatoria de la clase
            img = class_images[i % len(class_images)]
            img = np.expand_dims(img, 0)  # Expande dimensiones para el generador

            # Generar una imagen aumentada
            for batch in datagen.flow(img, batch_size=1):
                augmented_images.append(batch[0].astype(np.uint8))  # Agregar imagen aumentada
                augmented_labels.append(target_class)
                break  # Solo necesitamos una imagen por iteración

# Agregar las imágenes aumentadas al conjunto original
new_images_RGB = np.concatenate([new_images, np.array(augmented_images)])
new_labels = np.concatenate([new_labels, np.array(augmented_labels)])
new_images_RGB.shape

(4081, 28, 28, 3)

In [8]:

# new_images_Gris_v1 = []
# new_images_Gris_v2 = []
# new_images_Gris_v3 = []
# canal_rojo = new_images_RGB[:, :, :, 0]
# canal_verde = np.array(new_images_RGB[:, :, :, 1])
# canal_azul = np.array(new_images_RGB[:, :, :, 2])
# new_canal_v1 = ((canal_verde + canal_azul)/2).round(0)
# new_canal_v2 = ((canal_verde + canal_azul)).round(0)

# for img in new_images_RGB:
#     temp_gris = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     new_images_Gris_v1.append(temp_gris)
#     new_images_Gris_v2.append(temp_gris)
#     new_images_Gris_v3.append(temp_gris)

# new_images_Gris_v1 = np.stack((new_canal_v1, new_images_Gris_v1), axis=-1)
# new_images_Gris_v2 = np.stack((new_canal_v2, new_images_Gris_v2), axis=-1)
# new_images_Gris_v3 = np.stack((canal_rojo, new_images_Gris_v3), axis=-1)
# new_images_RC_v1 = np.stack((canal_rojo, new_canal_v1), axis=-1)
# new_images_RC_v2 = np.stack((canal_rojo, new_canal_v2), axis=-1)


### Pruebas con batch_size = 16
1: Con los hiperparametros actuales la propuesta 1 da mejor resolución con el conjunto completo de los 3 canales RGB 65%

2: Se descarta por completo la propuesta 2 de red neuronal por su bajo accuarycy en cualquiera de los casos 60%

3: Mejor caso RGB se mantiene igual65%

4: Mejor caso se mantiene con RGB65%

5: Mejor caso se mantiene con RGB 62%

los 3 mejores son 1,3,4

### Pruebas con batch_size = 32

1: Mejor caso RGB 64.5%

2: Mejor caso RGB 53%

3: Mejor caso RGB 61.2%

4: Mejor caso RGB 63.2%

5: Mejor caso RGB 63.7%

los 3 mejores son 1,3,4

In [9]:
# Red1_RGB = cnn.Propuesta_1(new_images_RGB, new_labels)
# Red1_RGB.CrearModelo()
# Red1_RGB.EvaluarModelo()

In [10]:
# Red1_Gris_v1 = cnn.Propuesta_1_RC(new_images_Gris_v1, new_labels)
# Red1_Gris_v1.CrearModelo()
# Red1_Gris_v1.EvaluarModelo()

In [11]:
# Red1_Gris_v2 = cnn.Propuesta_1_RC(new_images_Gris_v2, new_labels)
# Red1_Gris_v2.CrearModelo()
# Red1_Gris_v2.EvaluarModelo()

In [12]:
# Red1_Gris_v3 = cnn.Propuesta_1_RC(new_images_Gris_v3, new_labels)
# Red1_Gris_v3.CrearModelo()
# Red1_Gris_v3.EvaluarModelo()

In [13]:
# Red1_RC_v1 = cnn.Propuesta_1_RC(new_images_RC_v1, new_labels)
# Red1_RC_v1.CrearModelo()
# Red1_RC_v1.EvaluarModelo()

In [14]:
# Red1_RC_v2 = cnn.Propuesta_1_RC(new_images_RC_v2, new_labels)
# Red1_RC_v2.CrearModelo()
# Red1_RC_v2.EvaluarModelo()

In [15]:
# Red2_RGB = cnn.Propuesta_2(new_images_RGB, new_labels)
# Red2_RGB.CrearModelo()
# Red2_RGB.EvaluarModelo()

In [16]:
# Red2_Gris_v1 = cnn.Propuesta_2_RC(new_images_Gris_v1, new_labels)
# Red2_Gris_v1.CrearModelo()
# Red2_Gris_v1.EvaluarModelo()

In [17]:
# Red2_Gris_v2 = cnn.Propuesta_2_RC(new_images_Gris_v2, new_labels)
# Red2_Gris_v2.CrearModelo()
# Red2_Gris_v2.EvaluarModelo()

In [18]:
# Red2_Gris_v3 = cnn.Propuesta_2_RC(new_images_Gris_v3, new_labels)
# Red2_Gris_v3.CrearModelo()
# Red2_Gris_v3.EvaluarModelo()

In [19]:
# Red2_RC_v1 = cnn.Propuesta_2_RC(new_images_RC_v1, new_labels)
# Red2_RC_v1.CrearModelo()
# Red2_RC_v1.EvaluarModelo()

In [20]:
# Red2_RC_v2 = cnn.Propuesta_2_RC(new_images_RC_v2, new_labels)
# Red2_RC_v2.CrearModelo()
# Red2_RC_v2.EvaluarModelo()

In [21]:
# Red3_RGB = cnn.Propuesta_3(new_images_RGB, new_labels)
# Red3_RGB.CrearModelo()
# Red3_RGB.EvaluarModelo()

In [22]:
# Red3_Gris_v1 = cnn.Propuesta_3_RC(new_images_Gris_v1, new_labels)
# Red3_Gris_v1.CrearModelo()
# Red3_Gris_v1.EvaluarModelo()

In [23]:
# Red3_Gris_v2 = cnn.Propuesta_3_RC(new_images_Gris_v2, new_labels)
# Red3_Gris_v2.CrearModelo()
# Red3_Gris_v2.EvaluarModelo()

In [24]:
# Red3_Gris_v3 = cnn.Propuesta_3_RC(new_images_Gris_v3, new_labels)
# Red3_Gris_v3.CrearModelo()
# Red3_Gris_v3.EvaluarModelo()

In [25]:
# Red3_RC_v1 = cnn.Propuesta_3_RC(new_images_RC_v1, new_labels)
# Red3_RC_v1.CrearModelo()
# Red3_RC_v1.EvaluarModelo()

In [26]:
# Red3_RC_v2 = cnn.Propuesta_3_RC(new_images_RC_v2, new_labels)
# Red3_RC_v2.CrearModelo()
# Red3_RC_v2.EvaluarModelo()

In [27]:
# Red4_RGB = cnn.Propuesta_4(new_images_RGB, new_labels)
# Red4_RGB.CrearModelo()
# Red4_RGB.EvaluarModelo()

In [28]:
# Red4_Gris_v1 = cnn.Propuesta_4_RC(new_images_Gris_v1, new_labels)
# Red4_Gris_v1.CrearModelo()
# Red4_Gris_v1.EvaluarModelo()

In [29]:
# Red4_Gris_v2 = cnn.Propuesta_4_RC(new_images_Gris_v2, new_labels)
# Red4_Gris_v2.CrearModelo()
# Red4_Gris_v2.EvaluarModelo()

In [30]:
# Red4_Gris_v3 = cnn.Propuesta_4_RC(new_images_Gris_v3, new_labels)
# Red4_Gris_v3.CrearModelo()
# Red4_Gris_v3.EvaluarModelo()

In [31]:
# Red4_RC_v1 = cnn.Propuesta_4_RC(new_images_RC_v1, new_labels)
# Red4_RC_v1.CrearModelo()
# Red4_RC_v1.EvaluarModelo()

In [32]:
# Red4_RC_v2 = cnn.Propuesta_4_RC(new_images_RC_v2, new_labels)
# Red4_RC_v2.CrearModelo()
# Red4_RC_v2.EvaluarModelo()

In [33]:
# Red5_RGB = cnn.Propuesta_5(new_images_RGB, new_labels)
# Red5_RGB.CrearModelo()
# Red5_RGB.EvaluarModelo()

In [34]:
# Red5_Gris_v1 = cnn.Propuesta_5_RC(new_images_Gris_v1, new_labels)
# Red5_Gris_v1.CrearModelo()
# Red5_Gris_v1.EvaluarModelo()

In [35]:
# Red5_Gris_v2 = cnn.Propuesta_5_RC(new_images_Gris_v2, new_labels)
# Red5_Gris_v2.CrearModelo()
# Red5_Gris_v2.EvaluarModelo()

In [36]:
# Red5_Gris_v3 = cnn.Propuesta_5_RC(new_images_Gris_v3, new_labels)
# Red5_Gris_v3.CrearModelo()
# Red5_Gris_v3.EvaluarModelo()

In [37]:
# Red5_RC_v1 = cnn.Propuesta_5_RC(new_images_RC_v1, new_labels)
# Red5_RC_v1.CrearModelo()
# Red5_RC_v1.EvaluarModelo()

In [38]:
# Red5_RC_v2 = cnn.Propuesta_5_RC(new_images_RC_v2, new_labels)
# Red5_RC_v2.CrearModelo()
# Red5_RC_v2.EvaluarModelo()

In [39]:
test_dataset_28 = DataClass_28(split='test', download=download)
test_loader_28 = get_loader(dataset=test_dataset_28, batch_size=BATCH_SIZE)

paraPred_images_RGB = np.array(test_dataset_28.imgs)
paraPred_labels = np.array(test_dataset_28.labels)

# paraPred_images_Gris = []
# for img in paraPred_images_RGB:
#     temp_gris = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     paraPred_images_Gris.append(temp_gris)
# paraPred_images_Gris = np.array(paraPred_images_Gris)

# canal_rojo = paraPred_images_RGB[:, :, :, 0]
# canal_verde = np.array(paraPred_images_RGB[:, :, :, 1])
# canal_azul = np.array(paraPred_images_RGB[:, :, :, 2])
# new_canal = ((canal_verde + canal_azul) /2).round(0)
# paraPred_images_RC = np.stack((canal_rojo, new_canal), axis=-1)

Using downloaded and verified file: C:\Users\abiab\.medmnist\dermamnist.npz


In [45]:
RedConv = cnn.RedNeuronal(new_images_RGB, new_labels)
RedConv.EvaluarModelos()
valoresPredichos = RedConv.Predecir(paraPred_images_RGB)
precision = accuracy_score(paraPred_labels, valoresPredichos) * 100
print(f"Precisión: {precision:.2f}%")

Epoch 1/200


c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


184/184 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.1979 - loss: 2.7060 - val_accuracy: 0.1621 - val_loss: 2.3307
Epoch 2/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.2543 - loss: 2.2302 - val_accuracy: 0.1682 - val_loss: 2.3999
Epoch 3/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.2844 - loss: 2.1284 - val_accuracy: 0.2538 - val_loss: 2.0973
Epoch 4/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3349 - loss: 1.9937 - val_accuracy: 0.3700 - val_loss: 1.9016
Epoch 5/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3512 - loss: 1.9533 - val_accuracy: 0.3884 - val_loss: 1.8114
Epoch 6/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3753 - loss: 1.8783 - val_accuracy: 0.4251 - val_loss: 1.7753
Epoch 7/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3739 - loss: 1.8714 - val_accuracy: 0.4465 - val_loss: 1.7359
Epoch 8/200
184/184 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3877 - loss: 1.8172 - val_accuracy